<a href="https://colab.research.google.com/github/JanetteFu/Message_Detect/blob/main/Embedding_and_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
train_data = pd.read_csv('/content/sample_data/train_clean_data.csv')
test_data = pd.read_csv('/content/sample_data/test_clean_data.csv')

In [ ]:
print(test_data.isnull().sum())

Unnamed: 0    0
text          0
target        0
hashtag       0
dtype: int64


In [ ]:
train_data = train_data.dropna()

In [ ]:
train1 = train_data.loc[train_data['target'] == 1]
train0 = train_data.loc[train_data['target'] == 0]
test1 = test_data.loc[test_data['target'] == 1]
test0 = test_data.loc[test_data['target'] == 0]

In [ ]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

In [ ]:
train_list = train1.text.to_list()
train_list0 = train0.text.to_list()
train_target = train1.target.to_list()
train0_target = train0.target.to_list()
for i in range(0,2034):
  train_list.append(train_list0[i])
for i in range(0,2034):
  train_target.append(train0_target[i])

In [ ]:
train_list[2034]

'the console war is not real console choices do # perplexing'

In [ ]:
test_list = test1.text.to_list()
test_list0 = test0.text.to_list()
test_target = test1.target.to_list()
test0_target = test0.target.to_list()
for i in range(0,208):
  test_list.append(test_list0[i])

for i in range(0,208):
  test_target.append(test0_target[i])

In [ ]:
test_list[0:5]

['tragic white americans that prompts so many vote candidates who will slash their ow',
 '# allahsoil wwi wwii were both waged over oil # teambts # teamsuperjunior',
 'if you look at a problem and the first thing you thing of is color be it black or white you re pa of the problem',
 'racism is now front and center in trumpworld these kkk folk are now out in the open # kkk # trumpworld',
 'the revolution occurred just not how marx expected it to # teambts # teamsuperjunior']

In [ ]:
len(test_target)

416

Tokenize them into words

In [ ]:
wordlist = []
tokenize_df = pd.DataFrame(columns=['Tokenizer'])

In [ ]:
test_wordlist = []
tokenize_test = pd.DataFrame(columns=['Tokenizer'])

In [ ]:
for i in range(0,len(train_list)):
  temp = tknzr.tokenize(train_list[i])
  temp = [x for x in temp if x != '#']
  wordlist.append(temp)
  tokenize_df = tokenize_df.append({'Tokenizer': temp}, ignore_index=True)

In [ ]:
len(test_wordlist)

416

In [ ]:
for i in range(0,len(test_list)):
  temp = tknzr.tokenize(test_list[i])
  temp = [x for x in temp if x != '#']
  test_wordlist.append(temp)
  tokenize_test = tokenize_test.append({'Tokenizer': temp}, ignore_index=True)

In [ ]:
tokenize_df['target'] = train_target
tokenize_df.head()

,Tokenizer,target
0,"[not, to, sound, but, has, her, looks, to, fal...",1
1,"[zionism, is, a, form, of]",1
2,"[all, happy, for, lgbtfamily, but, what, i, do...",1
3,"[paul, beloy, is, still, an, issue, at, varyin...",1
4,"[this, is, really, telling, microaggressions, ...",1


In [ ]:
tokenize_test['target'][208:] = test0.target[0:208]
tokenize_test.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Tokenizer,target
0,"[tragic, white, americans, that, prompts, so, ...",1
1,"[allahsoil, wwi, wwii, were, both, waged, over...",1
2,"[if, you, look, at, a, problem, and, the, firs...",1
3,"[racism, is, now, front, and, center, in, trum...",1
4,"[the, revolution, occurred, just, not, how, ma...",1


In [ ]:
 tokenize_df.to_csv('df_tokenize.csv',encoding='utf-8')

In [ ]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Doc2Vec

In [ ]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec

model = Word2Vec(wordlist, size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")

NameError: ignored

#average score

In [ ]:
model = Word2Vec.load('/content/sample_data/word2vec.model')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Calculate for training_data

clean NA in tokenize_data

In [ ]:
import numpy as np
total_target = tokenize_df.target

In [ ]:
total_average = [0]*100
for i in range(0,len(wordlist)):
  sum_score = [0]*100
  for j in range(0,len(wordlist[i])):
    temp = model[wordlist[i][j]] 
    sum_score += temp
  average_score = sum_score/len(wordlist[i])
  total_average = np.vstack([total_average, average_score])
total_average = np.delete(total_average, (0), axis=0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [ ]:
total_average.shape

(4068, 100)

In [ ]:
np.savetxt("simple_average_train.csv", total_average, delimiter=",")

Calculating for testing data

In [ ]:
test_target = tokenize_test.target

In [ ]:
test_target

In [ ]:
test_average = [0]*100
for i in range(0,len(test_wordlist)):
  sum_score = [0]*100
  count = 0
  for j in range(0,len(test_wordlist[i])):
    if test_wordlist[i][j] in model.wv.vocab:
      temp = model[test_wordlist[i][j]] 
      sum_score += temp
      count += 1
  average_score = sum_score/count
  test_average = np.vstack([test_average, average_score])
test_average = np.delete(test_average, (0), axis=0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [ ]:
test_average.shape

(416, 100)

In [ ]:
np.savetxt("simple_average_test.csv", test_average, delimiter=",")

Weighting with tf idf


for training data

In [ ]:
s = wordlist[0] 
  
# using list comprehension 
listToStr0 = ' '.join([str(elem) for elem in s]) 
  
print(listToStr)

the console war is not real console choices do perplexing


In [ ]:
list = []
list.append(listToStr0)

In [ ]:
list = []
for i in range(0,len(wordlist)):
  s = wordlist[i] 
  
# using list comprehension 
  listToStr = ' '.join([str(elem) for elem in s]) 
  list.append(listToStr)

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.pipeline import Pipeline
import numpy as np

#instantiate CountVectorizer()
cv=CountVectorizer()

# this steps generates word counts for the words in your docs
word_count_vector=cv.fit_transform(list)

In [ ]:
word_count_vector.shape

(28763, 35127)

In [ ]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [ ]:
# count matrix
count_vector=cv.transform(list)

# tf-idf scores
tf_idf_vector=tfidf_transformer.transform(count_vector)

In [ ]:
feature_names = cv.get_feature_names()

#get tfidf vector for first document
first_document_vector=tf_idf_vector[0]

In [ ]:
tf_first_df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"])
tf_first_df.sort_values(by=["tfidf"],ascending=False)

,tfidf
console,0.763487
perplexing,0.381743
choices,0.295172
war,0.280534
real,0.226354
...,...
fuckwomenoffice,0.000000
fucku,0.000000
fucktrump,0.000000
fuckthisshit,0.000000


In [ ]:
tfidf_average = [0]*100
for i in range(0,len(wordlist)):
  sum_score = [0]*100
  for j in range(0,len(wordlist[i])):
    temp = model[wordlist[i][j]] 
    tf_idf_weight = tf_idf_vector[i].get(wordlist[i][j])
    sum_score += tf_idf_weight*temp
  tfidf_average = np.vstack([tfidf_average, sum_score])
tfidf_average = np.delete(tfidf_average, (0), axis=0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


AttributeError: ignored

for testing data

In [ ]:
test_count_vector=cv.transform(test_list)
# tf-idf scores
test_tf_idf_vector=tfidf_transformer.transform(test_count_vector)
test_tfidf_average = [0]*100
for i in range(0,len(test_wordlist)):
  sum_score = [0]*100
  for j in range(0,len(test_wordlist[i])):
    temp = model[test_wordlist[i][j]] 
    tf_idf_weight = test_tf_idf_vector[i].get(test_wordlist[i][j])
    sum_score += tf_idf_weight*temp
  test_tfidf_average = np.vstack([test_tfidf_average, sum_score])
test_tfidf_average = np.delete(test_tfidf_average, (0), axis=0)

build logistic regression classifier

In [ ]:
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression()
logisticRegr1 = LogisticRegression()

fit with average weighting

In [ ]:
print(train_target.isnull().sum())

AttributeError: ignored

In [ ]:
test_target

In [ ]:
logisticRegr.fit(total_average, train_target)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
len(test_target)

416

In [ ]:
predictions = logisticRegr.predict(test_average)
score = logisticRegr.score(test_average, test_target)
print(score)

0.5


In [ ]:
predictions[20:]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
unique, counts = np.unique(train_target, return_counts=True)
dict(zip(unique, counts))

{0: 2034, 1: 2034}

In [ ]:
weight_matrix1 = logisticRegr.coef_

In [ ]:
np.savetxt("simple_average_weight_matrix.csv", weight_matrix1, delimiter=",")

fit with tf_idf weighting

In [ ]:
logisticRegr1.fit(tfidf_average, total_target)
prediction1 = logisticRegr1.predict(test_tfidf_average)
score1 = logisticRegr1.score(test_tfidf_average, test_target)
print(score1)